# Training

This notebook goes through an example of how to use Faeyon for model fine-tuning. We will be using a ViT model and fine-tune it using a custom dataset. 

## Initialize model

We will use the pre-trained model -- . This model is available from the built-in configurations. 

In [1]:
from faeyon.models import Pipeline
from faeyon.models.tasks import ClassifyTask
from faeyon.training import Recipe
from faeyon.io import load

import torch
from torch import nn
from faeyon import X, Op

In [19]:
model = load("vit/vit-base-patch16-224", True)

TypeError: 'PosixPath' object is not subscriptable

In [6]:
pipeline = Pipeline(
    model=model,
    task=ClassifyTask(num_hidden=768, num_labels=1000, pooling=0)
)

In [7]:
x = torch.randn(1, 3, 224, 224)